In [ ]:
import pandas as pd
import transformers

## Reading dataset

In [ ]:
file_path= 'dataset_merged_iberlef.csv'
df= pd.read_csv(file_path)
# Filter by train dataset
train_df = df[df['source'] == 'train']
# Filter by dev dataset
dev_df = df[df['source'] == 'dev']

## Multiclass Classification

In [ ]:
target_names= ['0','1', '2', '3','4']
class_names= ['Not Hope','Generalized Hope', 'Realistic Hope','Unrealistic Hope', 'Sarcasm']

In [ ]:
def map_label(row):
    if row['multiclass']=='Not Hope':
        return 0
    elif row['multiclass']=='Generalized Hope':
        return 1
    elif row['multiclass']=='Realistic Hope':
        return 2
    elif row['multiclass']=='Unrealistic Hope':
        return 3
    elif row['multiclass']=='Sarcasm':
        return 4

train_df = train_df.assign(label= train_df.apply(map_label, axis=1))
dev_df = dev_df.assign(label= dev_df.apply(map_label, axis=1))

In [ ]:
from simpletransformers.classification import ClassificationModel


from simpletransformers.classification import (
    MultiLabelClassificationModel, MultiLabelClassificationArgs
)
import logging


logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [ ]:
def run_ModelTransformer(transformermodel, train, test, _target_names=[], _class_names=[]):
    # Create training data in the correct format for SimpleTransformers
    train_df = pd.DataFrame({
        'text': train['clean_text'],
        'labels': train['label']
    })

    # Ensure labels are integers (needed for SimpleTransformers)
    train_df['labels'] = train_df['labels'].astype(int)

    # For unlabeled test data, create a DataFrame with just the text
    test_df = pd.DataFrame({
        'text': test['clean_text']
    })


    # Create a TransformerModel
    model = ClassificationModel(
        transformermodel[0],
        transformermodel[1],
        num_labels=5,
        use_cuda=True,
        ignore_mismatched_sizes=True,
        args={
            'num_train_epochs': 1,
            'learning_rate': 1e-5,
            'max_seq_length': 64,
            'use_multiprocessing': False,
            'use_multiprocessing_for_evaluation': False,
            'overwrite_output_dir': True
        }
    )

    # Train the model with the properly formatted DataFrame
    model.train_model(train_df)

    # Using predict method
    predictions, raw_outputs = model.predict(test_df['text'].tolist())

    # Save predictions to the original test DataFrame and export
    test['predicted'] = predictions
    test.to_csv('predictions_multiclass_iberlef.csv', index=None)

    # Output information about the prediction process
    print("Predictions completed")
    print(f"Predicted {len(predictions)} samples.")

    # Return the predictions
    return {
        'predictions': predictions
    }

In [ ]:
run_ModelTransformer(['roberta', 'xlm-roberta-base'],train_df, dev_df, target_names, class_names)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

You are using a model of type xlm-roberta to instantiate a model of type roberta. This is not supported for all configurations of models and can yield errors.
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

/usr/local/lib/python3.11/dist-packages/simpletransformers/classification/classification_model.py:905: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():


  0%|          | 0/60 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/simpletransformers/classification/classification_model.py:2188: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():


Predictions completed for unlabeled test data.
Predicted 5990 samples.


{'predictions': array([0, 2, 2, ..., 0, 2, 0])}